In [1]:
import mne
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import cupy
import os
from pathlib import Path

In [2]:
data_dir = Path("data")

data_all = []
labels_all = []

os.environ['MNE_USE_CUDA'] = 'true' 
mne.utils.set_config('MNE_USE_CUDA', 'true') 
mne.cuda.init_cuda(ignore_config=True)  

Now using CUDA device 0
Enabling CUDA with 10.83 GB available memory


In [3]:
MIN_POINTS = 100  #Minimum number of time points per trial
TARGET_SAMPLING_RATE = 250
epoched_data_all=[]
for file_path in data_dir.glob("*.set"):
    file_name = file_path.stem
    if "PREP" not in file_name:
        number = int(file_name.split('_')[0])
        label = 1 if number % 2 == 1 else 0  # 1 = Parkinson's, 0 = Non-Parkinson's
        
        #Load the .set file
        raw = mne.io.read_raw_eeglab(file_path, preload=True)

        #Re-reference to the average reference
        raw.set_eeg_reference('average', projection=True)

        #Downsample to the target sampling rate
        raw.resample(TARGET_SAMPLING_RATE, npad="auto")

        #Bandpass filter
        raw.filter(1., 30., fir_design='firwin', n_jobs='cuda')

        #Apply ICA
        ica = mne.preprocessing.ICA(n_components=15, random_state=22, max_iter=1000, method='picard')
        ica.fit(raw)
        raw = ica.apply(raw)
        
        #Convert to numpy format
        data = raw.get_data()  # (n_channels, n_times)
        data = data.T
        
        #Epoch data
        epochs=mne.make_fixed_length_epochs(raw=raw,duration=30,preload=True)
        epochs_data=epochs.get_data()
        if data.shape[0] >= MIN_POINTS:
            data_all.append(data)
            labels_all.append(label)
            epoched_data_all.append(epochs_data)

print("Valid Trials:", len(data_all))



EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 825 samples (3.300 s)

Using CUDA for FFT FIR filtering
Fitting ICA to data using 66 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 1.7s.
Applying ICA to Raw instance
    Transform

C:\Users\Admin\AppData\Local\Temp\ipykernel_13948\1676369848.py:24: RuntimeWarning: Using n_components=15 (resulting in n_components_=15) may lead to an unstable mixing matrix estimation because the ratio between the largest (59) and smallest (8.6e-06) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 8
  ica.fit(raw)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 825 samples (3.300 s)

Using CUDA for FFT FIR filtering
Fitting ICA to data using 66 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 1.3s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 66 PCA components
Not setting metadata
5 matching events found
No baseline correction applied
Created an SSP operator (subspace dimens

In [ ]:
#minimum length of filtered data
target_length = min(data.shape[0] for data in data_all)

#Standardize the shape of each array in data_all
for i, data in enumerate(data_all):
    if data.shape[0] > target_length:
        #Shorten long arrays
        data_all[i] = data[:target_length, :]
    # elif data.shape[0] < target_length:
    #     #Lengthen short arrays
    #     padding = target_length - data.shape[0]
    #     data_all[i] = np.pad(data, ((0, padding), (0, 0)), mode="constant")

min_epochs = min(epochs_data.shape[0] for epochs_data in epoched_data_all)

#Shorten each trial epochs to the minimum number
for i, epochs_data in enumerate(epoched_data_all):
    epoched_data_all[i] = epochs_data[:min_epochs, :, :]

data_all = np.stack(data_all)#(n_trials, n_times, n_channels)
labels_all = np.array(labels_all)
epoched_data_all=np.stack(epoched_data_all)

print("Final dataset shape:",data_all.shape,"(trials, time points, channels)")
print("Labels shape:",labels_all.shape)
print("Epochs shape:",epoched_data_all.shape)

Final dataset shape: (79, 29838, 66) (trials, time points, channels)
Labels shape: (79,)
Epochs shape: (79, 3, 66, 7500)


In [5]:
import torch
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data_all, labels_all, test_size=0.2, random_state=42)
# X_train_tensor = torch.from_numpy(X_train).float()
# X_test_tensor = torch.from_numpy(X_test).float()
# y_train_tensor = torch.from_numpy(y_train).long()
# y_test_tensor = torch.from_numpy(y_test).long()


# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
# test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)


#For Epoched Data
epoch_X_train, epoch_X_test, epoch_y_train, epoch_y_test = train_test_split(epoched_data_all, labels_all, test_size=0.2, random_state=42)
# X_train_tensor = torch.from_numpy(X_train).float()
# X_test_tensor = torch.from_numpy(X_test).float()
# y_train_tensor = torch.from_numpy(y_train).long()
# y_test_tensor = torch.from_numpy(y_test).long()


# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
# test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import classification_report
def preprocess_data(X_train, X_test):
    #Flatten the data
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)
    
    #Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_flat)
    X_test_scaled = scaler.transform(X_test_flat)
    
    return X_train_scaled, X_test_scaled

def train_logistic_regression(X_train_scaled, y_train, X_test_scaled, y_test):
    lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')
    lr_model.fit(X_train_scaled, y_train)
    
    y_pred = lr_model.predict(X_test_scaled)
    print("\nLogistic Regression Results:")
    print(classification_report(y_test, y_pred))
    return lr_model

def train_logistic_regression_with_cv(X, y, cv_folds=5):
    lr_model = LogisticRegression(max_iter=1000, class_weight='balanced')
    
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    scores = cross_validate(
        lr_model, X, y, cv=cv, scoring=['accuracy', 'precision','f1','recall'],
        return_train_score=True, return_estimator=True
    )
    
    # print("Cross-Validation Results:")
    # for key in scores.keys():
    #     print(key,":",scores[key])
    #Return the model with the best performance on validation data
    best_model_index = np.argmax(scores['test_accuracy'])
    best_model = scores['estimator'][best_model_index]
    
    return best_model

In [8]:
def run_comparison(X_train, X_test, y_train, y_test):
    #Logistic Regression
    print("Training Logistic Regression...")
    X_train_scaled, X_test_scaled = preprocess_data(X_train, X_test)
    lr_model = train_logistic_regression(X_train_scaled, y_train, X_test_scaled, y_test)
    
    #Logistic Regression with cv
    print("Training Logistic Regression with cv...")
    X_combined = np.vstack((X_train_scaled, X_test_scaled))
    y_combined = np.hstack((y_train, y_test))
    best_model= train_logistic_regression_with_cv(X_combined, y_combined, cv_folds=5)
    y_pred = best_model.predict(X_test_scaled)
    print("\nTest Set Results:")
    print(classification_report(y_test, y_pred))
        
    
    return lr_model, best_model

lr_model, best_model = run_comparison(epoch_X_train, epoch_X_test, epoch_y_train, epoch_y_test)

Training Logistic Regression...

Logistic Regression Results:
              precision    recall  f1-score   support

           0       0.50      0.62      0.56         8
           1       0.50      0.38      0.43         8

    accuracy                           0.50        16
   macro avg       0.50      0.50      0.49        16
weighted avg       0.50      0.50      0.49        16

Training Logistic Regression with cv...

Test Set Results:
              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       1.00      0.88      0.93         8

    accuracy                           0.94        16
   macro avg       0.94      0.94      0.94        16
weighted avg       0.94      0.94      0.94        16

